# Battle of the Neighbourhoods week 1

## Problem statement:
Montes Claros is a medium-size city in the north of Minas Gerais, in Brazil. It's a cosmopolitan city, full of business investment oportunities. We would like to help invest in a new type of restaurant, one that hasn't got a lot of competition, so we're looking for types of restaurant that are more rare in the city, as well as good locations to place the new restaurant.

## Data gathering and evaluation:
We will use the Fourquare API to gather venue data around downtown Montes Claros, and plot the map to check the locations. We need to check the data for any venues that are not food related, and make graphs to quantify the different types of restaurant that are already available.

In [5]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import plotly.express as px

import json # library to handle JSON files
import requests # library to handle requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import seaborn as sns

print('Libraries imported.')

Libraries imported.


#### This next group cells is to get the coordinates for Montes Claros, and connect to the Fourquare API to get venue information for plotting.

In [6]:
address = 'Montes Claros'

geolocator = Nominatim(user_agent="moc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montes Claros are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montes Claros are -16.7273538, -43.8717676.


In [7]:
neighborhood_latitude = -16.7273538
neighborhood_longitude = -43.8717676

In [8]:
CLIENT_ID = '1NRHBF3WNXLYXSO3V1TT32IQULTYRIP3XB1MZX3PXUESNF4O'
CLIENT_SECRET = 'YJ134VZIWZNDJJOYT5U4C5XMGGKIE02VQNET0AD1EFRS0KPL'
VERSION = '20180604'

In [9]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [12]:
results = requests.get(url).json()

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
MOCnearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
MOCnearby_venues = MOCnearby_venues.loc[:, filtered_columns]

# filter the category for each row
MOCnearby_venues['venue.categories'] = MOCnearby_venues.apply(get_category_type, axis=1)

# clean columns
MOCnearby_venues.columns = [col.split(".")[-1] for col in MOCnearby_venues.columns]

MOCnearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,-16.728784,-43.871620
1,Rede Açai.com,Frozen Yogurt Shop,-16.725506,-43.872594
2,Point do Açaí,Dessert Shop,-16.726948,-43.867768
3,Center Pão,Bakery,-16.730044,-43.875340
4,Mapa de Minas,Bar,-16.725139,-43.872385


### Now that we have the venues, it's time to see them in the map of Montes Claros:

In [14]:
map_moc = folium.Map(location=[latitude, longitude], zoom_start=14)
for lat, lng, name in zip(MOCnearby_venues['lat'], MOCnearby_venues['lng'], MOCnearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moc) 
map_moc